# LAZY USER

In [ ]:
# LAZY USER

# Questi import fanno parte dell'ecosistema LangChain e servono per costruire agenti intelligenti che possono interagire con modelli OpenAI 
# (come GPT-4 o GPT-5) ed eseguire funzioni o tool esterni.
import os
os.environ["OPENAI_API_KEY"] = open("key.txt").read().strip() # run this code before engaging LLM model
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor # the tools needed to create a sort of chain with directives
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# =======================================
# 1) Creazione Tools
# =======================================


def create_reservation(name:str, date:str, time:str, people:int)-> str:
    """Crea una prenotazione e salva i dettagli in memory."""
    print('create_reservation called by Model')
    # (memory e' stato definito sotto, ma viene invocato prima che dentro il corpo di questa function quindi non dara' error)
    # riusciamo a chiamare le self.set_pending_action entro la classe SafeConversationMemory perche 'memory' e' un istanza di SafeConversationMemory
    # quando il modello decide di chiamare questo tool, allora chiamiamo memory.set_pending_action per inserire questo dato nella memory, cosi
    # il modello, grazie alle direttive del systemMessage(i punti 5 e 6), capisce che non deve ricreare l evento definito in questa tool (create_reservation) quando
    #  quando lo user da il consenso al modello di inserire il reservation nel calendario.), 
    #  
    memory.set_pending_action("create_reservation",{ # call the def set_pending_action that insert the pending_reservation state in memory, So the model understand what if we have a pending_task
        'name': name,
        'date': date,
        'time': time,
        'people': people 
    })

    return (f"Reservation created at '{name}' for {people} people on {date} at {time}."
            f"Do you want me to add this reservation to calendar?"
            )


def add_calendar_event(title:str, date:str, time:str)-> str:
    """Aggiunge l'evento al calendario. Se esiste una reservation pendente, la usa automaticamente."""
    print('add_calendar_event called by Model')

    # ✅ Prima di creare l’evento, controlla se c’è una reservation pendente

    pending = memory.get_pending_action('create_reservation') # controlla se c'e la pending_state creata dal tool 'create_reservation()' 
    if pending: # if chat_memory.metadata['pending_action'] has the name = create_reservation, clear # that once we add a new event add_calendar_event. Then we tell in the system about it in point 5 and 6
        # title = pending.get("name", title)
        # date = pending.get("date", date)
        # time = pending.get("time", time)

        # questa parte serve solo per essere sicuri che il modello abbia assegnato le corrette values
        # alle props di questa funzione (title:str, date:str, time:str). ma questo di solito avviene correttamente anche senza definire qui i values
        # questa e' solo una 'fallback' per essere sicuri che i dati inseriti nel calendario siano corretti, ma grazie al robusto systemMessage, questo
        # il modello genera (fino ad ora) sempre corretti values per queste props.(in production e' meglio ridefinire title,date,time come mostrato sopra)
        memory.clear_pending_action() # resetta dopo l'uso, per evitare chiamate doppie
    return f"Event {title} added at date {date} at time {time}."


# =======================================
# 2) define the Tools structure
# =======================================

from langchain.tools import StructuredTool 

calendar_tool = StructuredTool.from_function(
    func=add_calendar_event, 
    name="add_calendar_event",
    description="Add a event to calendar. Request title, date and time" # descrizioni utili al modello per creare i corretti args.
)

reservation_tool = StructuredTool.from_function(
    func=create_reservation, # define tool name
    name="create_reservation",
    description="Create a reservation (e.g., restaurant, barber, etc.). Requires name, date, time, and number of people." # descrizioni utili al modello per creare i corretti args.
)

tools = [calendar_tool,reservation_tool] 


# We need this line to describe in the system what tools he can access, so if there are many tools we dont need to write it in systemMessage one by one
# questa runzione ritorna il name , le props(time,date,people) e le description di ogni tool descritte in tools(sopra)
tool_descriptions = "\n".join(
    [f"- {t.name}({', '.join(t.args_schema.__annotations__.keys())}) -> {t.description}"
     for t in tools]
) 

# =======================================
# 3) Creazione SystemMessage
# =======================================

# 7. Only after explicit confirmation (like "yes", "ok", "all right"), call the appropriate tool.
lazy_user_prompt = f"""
You are **LazyUser**, a helpful but cautious assistant.

Your behavior must follow these principles:

1. **Intent Recognition:**  
   Always identify what the user intends to do (e.g., add, create, schedule, search, etc.).  

2. **Clarification First:**  
   If the user request is vague or missing details, politely ask clarifying questions before proceeding.  

3. **Confirmation Policy:**  
   If an action appears actionable, **always ask for confirmation** first before calling any tool.  
   Example: “Do you want me to add this to your calendar?”  

4. **Reservation Handling:**  
   After creating a reservation, always ask:  
   “Would you like me to add this reservation to your calendar?”  

5. **Pending Actions:**  
   If there is a pending reservation (you can infer this from memory or context), and the user confirms,  
   call the appropriate tool (like `add_calendar_event`) **without recreating the reservation**.  

6. **Reset Logic:**  
   When a completely new reservation request is made,  
   start a new reasoning flow and clear any previous pending reservations.  

7. **Tool Usage:**  
   Call it only after confirmation. Never call tools without explicit confirmation (e.g., “yes”, “ok”, “sure”).  

8. **Assistant Plan and Summarization:**  
     - BEFORE calling any tool: produce a short HUMAN-FACING plan sentence prefixed with "assistant_plan:" describing what you will do next (example: "assistant_plan: lets add this Event in the calender").
     - AFTER tool outputs, produce a clear HUMAN-FACING summary prefixed with "assistant_summary:" describing the result and next step (example: "assistant_summary: Event successfully added!").
     - NEVER autput chain-of-thought. The plan and summary are the only "thinking" visible to the user.

9. **Memory Awareness:**  
   Keep track of past interactions, confirmations, and pending actions.  
   Use this context to decide when to skip or repeat tool calls.

---

You have access to the following tools:
{tool_descriptions}

When the user's request clearly matches one of these actions, 
call the appropriate tool by reasoning about its name and description.
"""
#   Always explain what you are doing and why, using a natural, friendly tone. working one
# 6. **Show the assistent plan** -> BEFORE calling any tool: produce a short HUMAN-FACING plan sentence prefixed with "assistant_plan:" describing what you will do next (example: "assistant_plan: lets add this Event in the calender").
# 7. **Show summary** -> AFTER tool outputs, produce a clear HUMAN-FACING summary prefixed with "assistant_summary:" describing the result and next step (example: "assistant_summary: Event successfully added!").

# reason about which tool is appropriate and call it only after confirmation

# =======================================
#  Creazione del Prompt Template
# =======================================

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(lazy_user_prompt), # il nostro SystemMessage (le direttive che il modello deve seguire)
    MessagesPlaceholder(variable_name="chat_history"), # Prima di invocare il modello, MessagesPlaceholder prende le informazioni definite dall variabile chat_history definita dentro SafeConversationMemory(ConversationBufferMemory), per prendere migliori decisioni e aumentare lo user experience, cioe il modello ricorda cosa si e' detto prima per dare risposte senzate senza troppe spiegazioni
    MessagesPlaceholder(variable_name='pending_actions'), # nota che le variable_name devono matchare quelle definite nella classe SafeConversationMemory()
    HumanMessagePromptTemplate.from_template("{input}"), # Lo user_input. deve essere di formato HumanMessagePromptTemplate
    MessagesPlaceholder(variable_name="agent_scratchpad") # Inbuild key_word "agent_scratchpad". Serve all’agente interno di LangChain per “pensare a voce alta”.
    # agent_scratchpad e' il ragionamento interno o “traccia” del processo dell’agente (non mostrata all’utente finale). Credo che sia quello che
    # vediamo nel terminal che ci comunica ad esempio quando il tool viene invocato:
])


from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseMessage, HumanMessage

# ==========================================================================
# 2️⃣ Creazione della classe SafeConversationMemory(ConversationBufferMemory)

# cosa fa' questa classe?:
# 1) Context-aware pending state --> Il modello “vede” il pending_action nel contesto e può comportarsi di conseguenza.
# 2) Pulizia automatica messaggi --> Evita errori come "expected object but got string(puo' capitare quando il modello invoka una tool)",
#  oppure: expected string but got List/object(puo' capitare quando il content di BaseMessage e' un array, tipo:(content:['values here']) 
# invece di una stringa, tipo:(content:'values here'). contend deve essere una stringa per poter funzionare correttamente e non avere errors
# 3) Reset semplice --> Quando vuoi, puoi fare memory.clear_pending_action() per ripulire tutto (in questo caso quando il modello invoca il tool add_to_calendar)

# Abbiamo semplicemente inserito in memoria i pending state, cosi il modello quando creiamo una reservation, e poi inseriamo questa reservation 
# nel calendario, il modello non invoka dinuovo la stessa reservation tool(bug del modello). In poche parole risolviamo questo bug. 
# Grazie anche a questa classe possiamo dire al modello (nei punti 5 e 6 del systemMessage) di guardare dentro la memory se c'e' un pending_state
# e dare le direttive che risolvono questo bug(non chimare il tool create_reservation di nuovo quando invokiamo add_to_calender tool) 
# ==========================================================================

class SafeConversationMemory(ConversationBufferMemory):
    """
    Enhanced memory that:
    - Cleans malformed message content (list/dict -> string)
    - Keeps track of pending actions between user confirmations
    - Exposes pending actions to the model through context by adding it in memory, so in the systemMessage you tell the model to verify in memory
    - if there are pending actions
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self._pending_actions = {}  # store actions awaiting confirmation
      # self._pending_actions = {'create_reservation': {'name': 'da Alfredo', 'date': '21/04/2025', '21:00': 'time2', 'people': '2'}}

    # ============================================================
    # 🧠 1️⃣ Methods to manage pending actions
    # ============================================================

    def set_pending_action(self, name: str, data: dict): # we call this def from the tool def create_reservation(name,data)
        """Store a pending action waiting for user confirmation."""
        self._pending_actions[name] = data

    def get_pending_action(self, name: str): # controls if there are any pending actions when the model calls add_to_calender(name) Tool
        """Retrieve a specific pending action."""
        return self._pending_actions.get(name)

    def clear_pending_action(self, name: str = None):
        """Clear one or all pending actions."""
        if name:
            self._pending_actions.pop(name, None)
        else:
            self._pending_actions.clear()

    # ============================================================
    # 🧹 2️⃣ Override: cleaning + injecting context for the model
    # ============================================================

    def load_memory_variables(self, inputs):
        """Return cleaned chat history + pending actions as context."""
        raw = super().load_memory_variables(inputs)

        # ---- Clean malformed messages (lists/dicts converted to strings)
        if "chat_history" in raw:  # chat_history e' la key che abbiamo dato al nostro quando abbiamo definito 'memory'
            cleaned = []
            for msg in raw["chat_history"]:
                if isinstance(msg, BaseMessage): # BaseMessage(HumanMessage(content:'user_input') and AIMessage(content:'ai response')) is the rigth type for our model. Its the type the model require to work without errors.
                    # It is like that:
                    # {'chat_history': [HumanMessage(content='I need a reservation', additional_kwargs={}, response_metadata={}), AIMessage(content='Could you please clarify what type of reservation you need? For example, is it for a restaurant, hotel, or something else? Additionally, please provide any specific details like the date, time, and number of people if applicable.', additional_kwargs={}, response_metadata={})}
                    # so basically our memory stored in ConversationBufferMemory is stored in this way showed in HumanMessage and AIMessage
                    msg.content = str(msg.content) # turn (list, dict) content in str type to not throw errors
                    cleaned.append(msg)
                elif isinstance(msg, str): # if there are some 'fully string type' value(like not structured baseMessage data), instead of BaseMessage structure in our
                      # ConversationBufferMemory chat_history , then turn this string  to HumanMessage(). This 'string' bug can hapend when the model calls a tool, creating a 
                      # 'string with the name of the tool invoked', that the Model doesnt support and cannot parse.
                      #(HumanMessage is part of baseMessage for related to the user_input. there is the exemple right up in 'if isinstance(msg, BaseMessage)'). 
                     cleaned.append(HumanMessage(content=msg)) 
            raw["chat_history"] = cleaned # puliamo l object chat_history

        # ---- Add pending actions as extra conversational context. Questo previene chiamate al tool "reservation" quando invokiamo il tool add_to_calender
        # inseriamo in memoria che c'e un pending tool ogni volta che il tool "reservation" viene invocato, e nel systemMessage diciamo specificamente
        # al modello di non invocare nuovamente il tool "reservation" quando decidiamo di mettere questo "reservatio event" come evento nel tool "calender"
        # infine ogni volta che il modello inserisce l'evento sul tool calender, il pendig state ritorna vuoto, cosicche' il processo si ripete e 
        # non ci ripeta lo stesso problema
        if self._pending_actions: # if _pending_actions not empty, cioe' se ci sono dei pending_state 
            context_str = (
                f"(Pending actions currently stored: {self._pending_actions})" # questo va a' finire nella memory nel formato humaMessage come bediamo sotto
            )
        else:
            context_str = "(No pending actions stored.)"
        
        # aggiungiamo pending_actions in formato HumanMessage
        raw["pending_actions"] = [HumanMessage(content=context_str)] # trasformiamo il pending action in uno HumanMessage in modo che il formato sia
        # corretto da inserre nella memory e da essere compatimile con il formato del modello.
        # Ora  Il modello “vede” il pending_action nel contesto e può comportarsi di conseguenza (nei punti 5 e 6 del systemMessage diciamo al
        # di guardare i pending_state della memory e cosa fare se ce ne sono di attivi. 
        
        return raw  # ritorniamo la struttura di ConversationBufferMemory() piu' pulita e' con l aggiunt di pending_actions


# =======================================
# ✅  Creazione memoria. # qui invokiamo la classe SafeConversationMemory e passiamo i memory params al 'self'
# =======================================

memory = SafeConversationMemory( # istanza della classe SafeConversationMemory. Qui passiamo anche questi parametri definiti qua dentro alla classe
    # questa classe estende ConversationBufferMemory, con lo scopo di renderla piu' “intelligente” sicura.
    memory_key="chat_history",
    return_messages=True,
    input_key="input"  # 👈 forza un solo input
)


# =======================================
# ✅  Creazione modello
# =======================================

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)


# =======================================
# ✅  Creazione agente
# =======================================

agent = create_openai_functions_agent(
    llm=llm, 
    tools= tools, 
    prompt=prompt 
) 

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools= tools, 
    memory=memory,
    verbose=True,
    handle_parsing_errors=True, 
)



In [2]:

user_input = "I need a reservation" #"Aggiungi una sessione di stretching alle 7:30."
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers




> Entering new AgentExecutor chain...
Could you please provide me with the following details for the reservation?  
1. Name of the place (e.g., restaurant, barber, etc.)  
2. Date of the reservation  
3. Time of the reservation  
4. Number of people for the reservation  

> Finished chain.
Could you please provide me with the following details for the reservation?  
1. Name of the place (e.g., restaurant, barber, etc.)  
2. Date of the reservation  
3. Time of the reservation  
4. Number of people for the reservation  


In [3]:
user_input = "restaurant"
response = agent_executor.invoke({'input':user_input})
print(response['output'])



> Entering new AgentExecutor chain...
Could you please provide me with the following additional details for the restaurant reservation?  
1. Date of the reservation  
2. Time of the reservation  
3. Number of people for the reservation  

> Finished chain.
Could you please provide me with the following additional details for the restaurant reservation?  
1. Date of the reservation  
2. Time of the reservation  
3. Number of people for the reservation  


In [4]:

user_input = 'Name is "Da Alfredo", tomorrow at 21, 2 person', # This line strangely caused the baseMessage/HumanMessage(content=[Name is "Da Alfredo", tomorrow at 21, 2 person]) content value = list (error) 
# in 'memory' to be a List, and it caused an error. It has to be quite the same, but content value has to be string:    (content='Name is "Da Alfredo", tomorrow at 21, 2 person') content value = str (valid)
# Thats is one of the reasons we filter data in the class SafeConversationMemory, because ensure that content data is legit
response = agent_executor.invoke({'input':user_input})
print(response['output']) 
# memory.clear() # per capire se e' memory che da' error. model non capisce il contesto precedente se arrivato a questo punto cancelliamo la memory




> Entering new AgentExecutor chain...
Just to confirm, you would like to make a reservation at "Da Alfredo" for tomorrow at 21:00 for 2 people. Is that correct?

> Finished chain.
Just to confirm, you would like to make a reservation at "Da Alfredo" for tomorrow at 21:00 for 2 people. Is that correct?


In [5]:
user_input = 'yes please'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 
# OpenAI Function Agents di LangChain, specialmente quando si usa memoria (ConversationBufferMemory).
# significa che uno dei messaggi salvati in memoria non rispetta il formato richiesto da ChatOpenAI 
# (che si aspetta una lista di blocchi “structured message parts”, non semplici stringhe).
# In pratica:
# LangChain, quando aggiunge i messaggi precedenti dalla memoria (chat_history), trova un messaggio che non è nel formato corretto — tipicamente 
# perché è stato salvato come semplice 'str' invece che come BaseMessage (es. AIMessage o HumanMessage). infatti ConversationBufferMemory e  MessagesPlaceholder
# creano 'oggetti' di tipo:
#  {"role": "user", "content": "Ciao!"}, {type obj}
# {"role": "assistant", "content": "Ciao! Come posso aiutarti?"} {type obj} 
# e quando scriviamo il 'yes please', il modello risponde con una chiamata del tool, che risulta in una stringa(str) nella memory, che ci da' l error
# quando poi facciamo il parsing in json
print(response['output']) 
# {'chat_history': [HumanMessage(content='I need a reservation', additional_kwargs={}, response_metadata={}), AIMessage(content='Could you please clarify what type of reservation you need? For example, is it for a restaurant, hotel, or something else? Additionally, please provide any specific details like the date, time, and number of people if applicable.', additional_kwargs={}, response_metadata={}), HumanMessage(content='restaurant', additional_kwargs={}, response_metadata={}), AIMessage(content='Got it! I can help with a restaurant reservation. Could you please provide the following details?\n\n1. The name of the restaurant\n2. The date and time you want the reservation for\n3. The number of people for the reservation\n\nOnce I have that information, I can assist you further!', additional_kwargs={}, response_metadata={}), HumanMessage(content=['Name is "Da Alfredo", tomorrow at 21, 2 person'], additional_kwargs={}, response_metadata={}), AIMessage(content='Just to confirm, you would like to make a reservation at "Da Alfredo" for tomorrow at 21:00 for 2 people. \n\nIs that correct? If so, please confirm so I can proceed!', additional_kwargs={}, response_metadata={})]} Final raw
# notiamo che dentro questo output, abbiamo un content di tipo List:  HumanMessage(content=['Name is "Da Alfredo", tomorrow at 21, 2 person']. per non avere errors, il content deve contenere stringhe HumanMessage(content=('Name is "Da Alfredo", tomorrow at 21, 2 person'). Il modello OpenAI non accetta liste come content, ma solo stringhe o oggetti con chiave "type" e "text".
# Aggiorniamo la classe SafeConversationMemory in modo che trasformi automaticamente liste o dizionari in testo semplice prima di passarli al modello, così il modello riceve solo testo puro, e non list o object nel content, che scatenano l error





> Entering new AgentExecutor chain...

Invoking: `create_reservation` with `{'name': 'Da Alfredo', 'date': '2023-10-07', 'time': '21:00', 'people': 2}`
responded: assistant_plan: Let's create a reservation at "Da Alfredo" for tomorrow at 21:00 for 2 people.  
Now, I will proceed to create the reservation.

create_reservation called by Model
Reservation created at 'Da Alfredo' for 2 people on 2023-10-07 at 21:00.Do you want me to add this reservation to calendar?assistant_summary: Reservation successfully created at "Da Alfredo" for 2 people on October 7, 2023, at 21:00. Would you like me to add this reservation to your calendar?

> Finished chain.
assistant_summary: Reservation successfully created at "Da Alfredo" for 2 people on October 7, 2023, at 21:00. Would you like me to add this reservation to your calendar?


In [6]:
## model ask to add event to calendar
user_input = 'yes please'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 

# funziona bene fino a qui. il bug che chiamava nuovamente il tool create_reservation quando aggiungevamo questo reservation_event sul calendario 
# e' stato risolto grazie al fatto che abbiamo specificato nel system_message di non creare nuovamente un evento se quello precedente e ancora in corso
# e salvato in memoria(pending action)





> Entering new AgentExecutor chain...

Invoking: `add_calendar_event` with `{'title': 'Reservation at Da Alfredo', 'date': '2023-10-07', 'time': '21:00'}`


add_calendar_event called by Model
Event Reservation at Da Alfredo added at date 2023-10-07 at time 21:00.assistant_summary: The reservation at "Da Alfredo" has been successfully added to your calendar for October 7, 2023, at 21:00. If you need anything else, feel free to ask!

> Finished chain.
assistant_summary: The reservation at "Da Alfredo" has been successfully added to your calendar for October 7, 2023, at 21:00. If you need anything else, feel free to ask!


In [ ]:
# the model do recognize general input, that requires no tool calling 
user_input = 'tnx chat'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 



> Entering new AgentExecutor chain...
You're welcome! If you need any more assistance in the future, feel free to reach out. Have a great day!

> Finished chain.
You're welcome! If you need any more assistance in the future, feel free to reach out. Have a great day!


In [8]:
# testing again to create another reservation to se if the previus one creared the pending actions:

# Funziona bene anche da qui fino a fine input, cioe' quando creiamo il nuovo reservation e aggiungiamo nuovamente la reservation event nel calendario, 
# il bug non si presenta. A dire il vero il giorno prima si era ripresentato, ma oggi no', quindi si puo' dire che funziona bene, ma puo' ancora 
# presentare il bug della chiamata extra della tool create_reservation. ma per il momento va piu' che bene cosi'.
user_input = 'i need another reservation' # come abbiamo scritto prima nel codice, quando il modello aggiunge un evento nel calendario, la pendig_state value viene azzerata, e quindi il modello  
response = agent_executor.invoke({'input':user_input})  
print(response['output']) 



> Entering new AgentExecutor chain...
Could you please provide me with the following details for the new reservation?  
1. Name of the place (e.g., restaurant, barber, etc.)  
2. Date of the reservation  
3. Time of the reservation  
4. Number of people for the reservation  

> Finished chain.
Could you please provide me with the following details for the new reservation?  
1. Name of the place (e.g., restaurant, barber, etc.)  
2. Date of the reservation  
3. Time of the reservation  
4. Number of people for the reservation  


In [9]:

user_input = 'barber'
response = agent_executor.invoke({'input':user_input}) 
print(response['output']) 




> Entering new AgentExecutor chain...
Could you please provide me with the following additional details for the barber reservation?  
1. Date of the reservation  
2. Time of the reservation  
3. Number of people for the reservation  

> Finished chain.
Could you please provide me with the following additional details for the barber reservation?  
1. Date of the reservation  
2. Time of the reservation  
3. Number of people for the reservation  


In [10]:
user_input = 'Name is "Saho", tomorrow at 16', # Maybe it doenst understantd tomorrow, and it ll fail 
# pass the props date to calender function
response = agent_executor.invoke({'input':user_input})
print(response['output']) 




> Entering new AgentExecutor chain...
Just to confirm, you would like to make a reservation at "Saho" for tomorrow at 16:00. How many people will be in the reservation?

> Finished chain.
Just to confirm, you would like to make a reservation at "Saho" for tomorrow at 16:00. How many people will be in the reservation?


In [11]:
user_input = 'yes please'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 



> Entering new AgentExecutor chain...
It looks like I need the number of people for the reservation at "Saho" tomorrow at 16:00. Could you please provide that?

> Finished chain.
It looks like I need the number of people for the reservation at "Saho" tomorrow at 16:00. Could you please provide that?


In [12]:
user_input = 'just me'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 



> Entering new AgentExecutor chain...
Just to confirm, you would like to make a reservation at "Saho" for 1 person tomorrow at 16:00. Is that correct?

> Finished chain.
Just to confirm, you would like to make a reservation at "Saho" for 1 person tomorrow at 16:00. Is that correct?


In [ ]:
# give the persmission to the model to create the reservation
user_input = 'yes please'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 



> Entering new AgentExecutor chain...

Invoking: `create_reservation` with `{'name': 'Saho', 'date': '2023-10-07', 'time': '16:00', 'people': 1}`


create_reservation called by Model
Reservation created at 'Saho' for 1 people on 2023-10-07 at 16:00.Do you want me to add this reservation to calendar?The reservation has been successfully created at "Saho" for 1 person tomorrow at 16:00. Would you like me to add this reservation to your calendar?

> Finished chain.
The reservation has been successfully created at "Saho" for 1 person tomorrow at 16:00. Would you like me to add this reservation to your calendar?


In [ ]:
# give the persmission to the model to add to calendar

user_input = 'yes please'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 



> Entering new AgentExecutor chain...

Invoking: `add_calendar_event` with `{'title': 'Reservation at Saho', 'date': '2023-10-07', 'time': '16:00'}`


add_calendar_event called by Model
Event Reservation at Saho added at date 2023-10-07 at time 16:00.assistant_summary: The reservation at "Saho" for 1 person has been successfully added to your calendar for October 7, 2023, at 16:00. If you need anything else, feel free to ask!

> Finished chain.
assistant_summary: The reservation at "Saho" for 1 person has been successfully added to your calendar for October 7, 2023, at 16:00. If you need anything else, feel free to ask!


In [15]:
# greatings model
user_input = 'tnx chat'
response = agent_executor.invoke({'input':user_input}) # errore 400 al “yes please” è un caso molto comune quando si lavora con gli 

print(response['output']) 



> Entering new AgentExecutor chain...
You're welcome! If you need any more help in the future, just let me know. Have a great day!

> Finished chain.
You're welcome! If you need any more help in the future, just let me know. Have a great day!
